In [ ]:
!pip install tensorflow

In [ ]:
import os
import numpy as np
import pandas as pd
import wave
from google.colab import drive
import tensorflow.compat.v1 as tf
import zipfile
import librosa
from keras import layers
import math
import shutil
from tqdm import tqdm

prefix = '/content/drive/MyDrive/DAIC-WOZ'
assert os.path.exists(prefix), "Data folder not found in Google Drive."

In [ ]:
# Load CSV files
train_split_df = pd.read_csv(os.path.join(prefix, 'train_split_Depression_AVEC2017.csv'))
test_split_df = pd.read_csv(os.path.join(prefix, 'dev_split_Depression_AVEC2017.csv'))

train_split_num = train_split_df['Participant_ID'].tolist()
test_split_num = test_split_df['Participant_ID'].tolist()
train_split_label = train_split_df['PHQ8_Binary'].tolist()
test_split_label = test_split_df['PHQ8_Binary'].tolist()

# Load queries.txt
with open(os.path.join(prefix, 'queries.txt')) as f:
    queries = f.readlines()

In [ ]:
def identify_topics(sentence):
    for query in queries:
        query = query.strip('\n')
        sentence = sentence.strip('\n')
        if query == sentence:
            return True
    return False

cluster_size = 16

def wav2vlad(wave_data, sr):
    global cluster_size
    signal = wave_data
    melspec = librosa.feature.melspectrogram(y=signal, n_mels=80, sr=sr).T
    melspec = np.log(np.maximum(1e-6, melspec))
    feature_size = melspec.shape[1]
    max_samples = melspec.shape[0]
    output_dim = cluster_size * 16
    feat = NetVLAD(feature_size=feature_size, max_samples=max_samples, \
                            cluster_size=cluster_size, output_dim=output_dim) \
                                (tf.convert_to_tensor(melspec))
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        r = feat.numpy()
    return r

class NetVLAD(layers.Layer):
    """Creates a NetVLAD class.
    """
    def __init__(self, feature_size, max_samples, cluster_size, output_dim, **kwargs):

        self.feature_size = feature_size
        self.max_samples = max_samples
        self.output_dim = output_dim
        self.cluster_size = cluster_size
        super(NetVLAD, self).__init__(**kwargs)

    def build(self, input_shape):
    # Create a trainable weight variable for this layer.
        self.cluster_weights = self.add_weight(name='kernel_W1',
                                      shape=(self.feature_size, self.cluster_size),
                                      initializer=tf.random_normal_initializer(stddev=1 / math.sqrt(self.feature_size)),
                                      trainable=True)
        self.cluster_biases = self.add_weight(name='kernel_B1',
                                      shape=(self.cluster_size,),
                                      initializer=tf.random_normal_initializer(stddev=1 / math.sqrt(self.feature_size)),
                                      trainable=True)
        self.cluster_weights2 = self.add_weight(name='kernel_W2',
                                      shape=(1,self.feature_size, self.cluster_size),
                                      initializer=tf.random_normal_initializer(stddev=1 / math.sqrt(self.feature_size)),
                                      trainable=True)
        self.hidden1_weights = self.add_weight(name='kernel_H1',
                                      shape=(self.cluster_size*self.feature_size, self.output_dim),
                                      initializer=tf.random_normal_initializer(stddev=1 / math.sqrt(self.cluster_size)),
                                      trainable=True)

        super(NetVLAD, self).build(input_shape)  # Be sure to call this at the end

    def call(self, reshaped_input):
        """Forward pass of a NetVLAD block.

        Args:
        reshaped_input: If your input is in that form:
        'batch_size' x 'max_samples' x 'feature_size'
        It should be reshaped in the following form:
        'batch_size*max_samples' x 'feature_size'
        by performing:
        reshaped_input = tf.reshape(input, [-1, features_size])

        Returns:
        vlad: the pooled vector of size: 'batch_size' x 'output_dim'
        """
        """
        In Keras, there are two way to do matrix multiplication (dot product)
        1) K.dot : AxB -> when A has batchsize and B doesn't, use K.dot
        2) tf.matmul: AxB -> when A and B both have batchsize, use tf.matmul

        Error example: Use tf.matmul when A has batchsize (3 dim) and B doesn't (2 dim)
        ValueError: Shape must be rank 2 but is rank 3 for 'net_vlad_1/MatMul' (op: 'MatMul') with input shapes: [?,21,64], [64,3]

        tf.matmul might still work when the dim of A is (?,64), but this is too confusing.
        Just follow the above rules.
        """
        activation = tf.matmul(reshaped_input, self.cluster_weights)

        activation += self.cluster_biases

        activation = tf.nn.softmax(activation)

        activation = tf.reshape(activation,
                [-1, self.max_samples, self.cluster_size])

        a_sum = tf.reduce_sum(activation,-2,keep_dims=True)

        a = tf.multiply(a_sum,self.cluster_weights2)

        activation = tf.transpose(activation,perm=[0,2,1])

        reshaped_input = tf.reshape(reshaped_input,[-1,
            self.max_samples, self.feature_size])

        vlad = tf.matmul(activation,reshaped_input)
        vlad = tf.transpose(vlad,perm=[0,2,1])
        vlad = tf.subtract(vlad,a)
        vlad = tf.nn.l2_normalize(vlad,1)
        vlad = tf.reshape(vlad,[-1, self.cluster_size*self.feature_size])
        vlad = tf.nn.l2_normalize(vlad,1)
        vlad = tf.matmul(vlad, self.hidden1_weights)

        return vlad

    def compute_output_shape(self, input_shape):
        return tuple([None, self.output_dim])

def extract_features(number, extract_zip = False):
    # Extract ZIP file
    if extract_zip:
        zip_path = os.path.join(prefix, f'{number}_P.zip')
        extract_dir = os.path.join(prefix, f'{number}_P')

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

    # Load the transcript and audio data
    transcript_path = os.path.join(prefix, f'{number}_P/{number}_TRANSCRIPT.csv')
    audio_path = os.path.join(prefix, f'{number}_P/{number}_AUDIO.wav')

    transcript = pd.read_csv(transcript_path, sep='\t').fillna('')
    wavefile = wave.open(audio_path, 'r')
    sr = wavefile.getframerate()
    nframes = wavefile.getnframes()
    wave_data = np.frombuffer(wavefile.readframes(nframes), dtype=np.short)

    response = ''
    audio_feats = []
    text_feats = []
    signal = []

    for t in transcript.itertuples():
        # Check if the question is asked by "Ellie" and contains specific phrases
        if getattr(t, 'speaker') == 'Ellie' and (identify_topics(getattr(t, 'value')) or 'i think i have asked everything' in getattr(t, 'value')):
            if len(signal) == 0:
                continue
            audio_feats.append(wav2vlad(signal, sr))
            text_feats.append(response)
            response = ''
            signal = []
        elif getattr(t, 'speaker') == 'Participant':
            if 'scrubbed_entry' in getattr(t, 'value'):
                continue
            start_time = int(getattr(t, 'start_time') * sr)
            stop_time = int(getattr(t, 'stop_time') * sr)
            response += ' ' + getattr(t, 'value')
            signal = np.hstack((signal, wave_data[start_time:stop_time].astype(float)))

    print(f'{number}_P feature done size {np.shape(audio_feats)}')

    # Clean up extracted files to free up space
    if extract_zip:
        shutil.rmtree(extract_dir)
    return audio_feats, text_feats

In [ ]:
save_dir = os.path.join('/content/drive/MyDrive', 'DAIC-Features')

# Ensure the directory exists; if not, create it
os.makedirs(save_dir, exist_ok=True)

audio_features_train, text_features_train, targets_train = [], [], []
audio_features_test, text_features_test, targets_test = [], [], []

# Process training set
print("Processing training data...")
for index in tqdm(range(len(train_split_num)), desc="Training Set"):
    audio_feats, text_feats = extract_features(train_split_num[index])
    audio_features_train.append(audio_feats)
    text_features_train.append(text_feats)
    targets_train.append(train_split_label[index])

print("Saving training data to Google Drive...")
# Save each audio feature and text feature list separately with participant IDs as keys
np.savez(
    os.path.join(save_dir, 'train_audio_clf.npz'),
    **{f'audio_{train_split_num[i]}': feat for i, feat in enumerate(audio_features_train)}
)
np.savez(
    os.path.join(save_dir, 'train_text_clf.npz'),
    **{f'text_{train_split_num[i]}': feat for i, feat in enumerate(text_features_train)}
)
np.savez(
    os.path.join(save_dir, 'train_label_clf.npz'),
    **{f'label_{train_split_num[i]}': targets_train[i] for i in range(len(targets_train))}
)

# Process test set
print("Processing test data...")
for index in tqdm(range(len(test_split_num)), desc="Test Set"):
    audio_feats, text_feats = extract_features(test_split_num[index])
    audio_features_test.append(audio_feats)
    text_features_test.append(text_feats)
    targets_test.append(test_split_label[index])

print("Saving test data to Google Drive...")
# Save each audio feature and text feature list separately with participant IDs as keys
np.savez(
    os.path.join(save_dir, 'test_audio_clf.npz'),
    **{f'audio_{test_split_num[i]}': feat for i, feat in enumerate(audio_features_test)}
)
np.savez(
    os.path.join(save_dir, 'test_text_clf.npz'),
    **{f'text_{test_split_num[i]}': feat for i, feat in enumerate(text_features_test)}
)
np.savez(
    os.path.join(save_dir, 'test_label_clf.npz'),
    **{f'label_{test_split_num[i]}': targets_test[i] for i in range(len(targets_test))}
)

Processing training data...


Training Set:   1%|          | 1/107 [00:07<13:00,  7.36s/it]

303_P feature done size (34, 1, 256)


Training Set:   2%|▏         | 2/107 [00:13<11:55,  6.82s/it]

304_P feature done size (46, 1, 256)


...

489_P feature done size (50, 1, 256)


Test Set:  97%|█████████▋| 34/35 [06:39<00:12, 12.78s/it]

490_P feature done size (50, 1, 256)


Test Set: 100%|██████████| 35/35 [06:50<00:00, 11.74s/it]

492_P feature done size (41, 1, 256)
Saving test data to Google Drive...


In [ ]:
# Sample Usage
save_dir = os.path.join('/content/drive/MyDrive', 'DAIC-Features')

loaded_audio_data = np.load(os.path.join(save_dir, 'train_audio_clf.npz'))
audio_feature_303 = loaded_audio_data['audio_303']  # Access data by participant ID

loaded_audio_data = np.load(os.path.join(save_dir, 'train_text_clf.npz'))
text_feature_303 = loaded_audio_data['text_303']  # Access data by participant ID

loaded_audio_data = np.load(os.path.join(save_dir, 'train_label_clf.npz'))
label_303 = loaded_audio_data['label_303']  # Access data by participant ID

In [ ]:
audio_feature_303.shape

(34, 1, 256)

In [ ]:
text_feature_303[:20]

array([" okay how 'bout yourself", ' here in california yeah',
       " oh well that it's big and broad there's a lot to do a lot of um um job opportunities than other states um pretty much that it's big and there's a lot you can do here",
       " traffic um maybe the violence rate bad news even though you know you wanna know what's going on in your environment but you still have to watch it so you can you know look out to see what's going on on a daily basis",
       ' sociology i have a um two year degree in liberal arts but my major was sociology',
       " no i'm actually an m_t_a bus operator",
       " well since metro is steady growing um my dream job is to move up in the company and i'm about to actually start some classes for supervisory next week so um 'cause they look to promote within the company so that's the good thing there's a lot of opportunities at metro where you can take classes and they'll pay for your tuition and things of that nature thanks",
       " well it's 

In [ ]:
label_303

array(0)